# Data Preprocessing

For: Pao Pao and Suhut Mickey Lin

This file takes in non-text data files in the data folder and aggregate it into train and test files for the *model_*.ipynb* files. This is the most time sensitive. I'll make sure this is done before Wednesday 23rd Apr 2025 (I know that this is extremely late sorry). For now the code below generates a test file for yall to test out the code for the models. The performance will probably be bad but as long as the code runs that is good enough.

In [20]:
import pandas as pd
from scipy.stats.mstats import winsorize
import numpy as np

from companies import TICKERS

from sklearn.preprocessing import MinMaxScaler

## Econ data

In [2]:
econ_path = "data/econ_data"

df_gdp = pd.read_csv(f"{econ_path}/GDP.csv")  # https://fred.stlouisfed.org/series/GDP (QUARTERLY)
df_gdp = df_gdp.rename(columns={"GDP":"GDP (BILLIONS OF DOLLARS)"})
df_gdp["GDP CHANGE (-1 to 1)"] = df_gdp["GDP (BILLIONS OF DOLLARS)"].pct_change()
df_gdp['observation_date'] = pd.to_datetime(df_gdp['observation_date'])
df_gdp['quarter'] = df_gdp['observation_date'].dt.to_period('Q')

df_unrate = pd.read_csv(f"{econ_path}/UNRATE.csv")  # https://fred.stlouisfed.org/series/UNRATE (MONTHLY)
df_unrate = df_unrate.rename(columns={"UNRATE":"UNEMPLOYMENT RATE (0 to 1)"})
df_unrate["UNEMPLOYMENT RATE (0 to 1)"] = df_unrate["UNEMPLOYMENT RATE (0 to 1)"] / 100
df_unrate['observation_date'] = pd.to_datetime(df_unrate['observation_date'])

df_dprime = pd.read_csv(f"{econ_path}/DPRIME.csv")  # https://fred.stlouisfed.org/series/DPRIME (DAILY)
df_dprime = df_dprime.rename(columns={"DPRIME": "PRIME LOAN RATE (0 to 1)"})
df_dprime["PRIME LOAN RATE (0 to 1)"] = df_dprime["PRIME LOAN RATE (0 to 1)"] / 100
df_dprime['observation_date'] = pd.to_datetime(df_dprime['observation_date'])

df_deposit = pd.read_csv(f"{econ_path}/DEPOSITS.csv")  # https://fred.stlouisfed.org/series/DPSACBW027SBOG (WEEKLY)
df_deposit = df_deposit.rename(columns={"DPSACBW027SBOG":"DEPOSITS (BILLIONS OF DOLLARS)"})
df_deposit['observation_date'] = pd.to_datetime(df_deposit['observation_date'])

df_inflation = pd.read_csv(f"{econ_path}/INFLATION.csv")  # https://fred.stlouisfed.org/series/CORESTICKM159SFRBATL (MONTHLY)
df_inflation = df_inflation.rename(columns={"CORESTICKM159SFRBATL":"CONSUMER PRICE INDEX (0 to 1)"})
df_inflation["CONSUMER PRICE INDEX (0 to 1)"] = df_inflation["CONSUMER PRICE INDEX (0 to 1)"] / 100
df_inflation['observation_date'] = pd.to_datetime(df_inflation['observation_date'])

df_savings = pd.read_csv(f"{econ_path}/SAVINGS.csv")  # https://fred.stlouisfed.org/series/W207RC1Q156SBEA (Quarterly)
df_savings = df_savings.rename(columns={"W207RC1Q156SBEA":"SAVINGS PER GROSS INCOME (-1 to 1)"})
df_savings["SAVINGS PER GROSS INCOME (-1 to 1)"] = df_savings["SAVINGS PER GROSS INCOME (-1 to 1)"] / 100
df_savings['observation_date'] = pd.to_datetime(df_savings['observation_date'])
df_savings['quarter'] = df_savings['observation_date'].dt.to_period('Q')


In [3]:
gpd_change_by_quarter = df_gdp[["quarter", "GDP CHANGE (-1 to 1)"]]

df_unrate['quarter'] = df_unrate['observation_date'].dt.to_period('Q')
avg_unemployment_by_quarter = df_unrate.groupby('quarter')['UNEMPLOYMENT RATE (0 to 1)'].mean().reset_index()

df_dprime['quarter'] = df_dprime['observation_date'].dt.to_period('Q')
avg_primerate_by_quarter = df_dprime.groupby('quarter')['PRIME LOAN RATE (0 to 1)'].mean().reset_index()

df_deposit['quarter'] = df_deposit['observation_date'].dt.to_period('Q')
avg_deposit_by_quarter = df_deposit.groupby('quarter')['DEPOSITS (BILLIONS OF DOLLARS)'].mean().reset_index()
avg_deposit_by_quarter["DEPOSITS CHANGE (-1 to 1)"] = avg_deposit_by_quarter["DEPOSITS (BILLIONS OF DOLLARS)"].pct_change()
avg_deposit_by_quarter = avg_deposit_by_quarter[["quarter", "DEPOSITS CHANGE (-1 to 1)"]]

df_inflation['quarter'] = df_inflation['observation_date'].dt.to_period('Q')
avg_inflation_by_quarter = df_inflation.groupby('quarter')['CONSUMER PRICE INDEX (0 to 1)'].mean().reset_index()

net_savings_pecent_by_quarter = df_savings[["quarter", "SAVINGS PER GROSS INCOME (-1 to 1)"]]

In [4]:
econ_dfs = [gpd_change_by_quarter, avg_unemployment_by_quarter, avg_primerate_by_quarter, avg_deposit_by_quarter, avg_inflation_by_quarter, net_savings_pecent_by_quarter]

econ_merged_df = econ_dfs[0]
for df in econ_dfs[1:]:
    econ_merged_df = pd.merge(econ_merged_df, df, on="quarter", how="outer")

In [7]:
min_q = "2000Q1"
max_q = "2024Q4"

In [8]:
merged_df_2000_to_2024 = econ_merged_df.loc[(econ_merged_df['quarter'] >= min_q) & (econ_merged_df['quarter'] <= max_q)]

In [10]:
merged_df_2000_to_2024.head()  # will need to do min-max scale later

,quarter,GDP CHANGE (-1 to 1),UNEMPLOYMENT RATE (0 to 1),PRIME LOAN RATE (0 to 1),DEPOSITS CHANGE (-1 to 1),CONSUMER PRICE INDEX (0 to 1),SAVINGS PER GROSS INCOME (-1 to 1)
212,2000Q1,0.010304,0.040333,0.086944,0.019192,0.023993,0.063
213,2000Q2,0.024549,0.039333,0.092500,0.021062,0.025565,0.060
214,2000Q3,0.006874,0.040000,0.095000,0.020524,0.028283,0.060
215,2000Q4,0.011395,0.039000,0.095000,0.010801,0.029207,0.054
216,2001Q1,0.003305,0.042333,0.086210,0.024485,0.029400,0.055


In [ ]:
merged_df_2000_to_2024 = merged_df_2000_to_2024.rename(columns={"quarter": "datacqtr"})
merged_df_2000_to_2024["q"]

## Fundamentals - first file

In [12]:
fundamentals_path = "data/fundamentals"

df_fundamentals = pd.read_csv(f"{fundamentals_path}/fundamentals_banking.csv")
df_fundamentals["cik"] = df_fundamentals["cik"].astype(str)

/var/folders/78/9j91h4q516z7774553xzp19m0000gn/T/ipykernel_94817/2851958132.py:3: DtypeWarning: Columns (26,650,657,668,669,679) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fundamentals = pd.read_csv(f"{fundamentals_path}/fundamentals_banking.csv")


In [13]:
interested_cols = ["niitq", "nimq", "ncoq", "cheq", "niq"]  # finrevq
name_map = {"niitq": "Net Interest Income", "nimq": "Net Interest Margin", "ncoq": "Net Charge-Offs", "cheq": "Cash and Short-Term Investments", "niq": "Net Income"}
key_cols = ["datacqtr", "datadate", "rdq", "tic", "cik",]

In [14]:
# Check if any columns cant be found
for col in interested_cols:
    if col not in df_fundamentals.columns:
        print(col)

In [15]:
df_fundamentals_simplified = df_fundamentals[key_cols + interested_cols]
df_fundamentals_simplified = df_fundamentals_simplified.rename(columns=name_map)

In [16]:
nan_columns = df_fundamentals_simplified.columns[df_fundamentals_simplified.isna().all()].tolist()

print("Columns with only NaN values:", nan_columns)

Columns with only NaN values: []


In [18]:
df_fundamentals_simplified_winsorize = df_fundamentals_simplified.copy()
limits_list = [(0.01, 0.01), (0.01, 0.1), (0.01, 0.01), (0.01, 0.01), (0.01, 0.01)]
for col, limits in zip(name_map.values(), limits_list):
    df_fundamentals_simplified_winsorize[col] = winsorize(df_fundamentals_simplified_winsorize[col], limits=limits, nan_policy="omit")

In [21]:
df_fundamentals_simplified_winsorize["Net Interest Income"] = np.sign(df_fundamentals_simplified_winsorize["Net Interest Income"]) * np.log1p(np.abs(df_fundamentals_simplified_winsorize["Net Interest Income"]))
df_fundamentals_simplified_winsorize["Cash and Short-Term Investments"] = np.sign(df_fundamentals_simplified_winsorize["Cash and Short-Term Investments"]) * np.log1p(np.abs(df_fundamentals_simplified_winsorize["Cash and Short-Term Investments"]))
df_fundamentals_simplified_winsorize["Net Income"] = np.sign(df_fundamentals_simplified_winsorize["Net Income"]) * np.log1p(np.abs(df_fundamentals_simplified_winsorize["Net Income"]))
df_fundamentals_simplified_winsorize["Net Charge-Offs"] = np.sign(df_fundamentals_simplified_winsorize["Net Charge-Offs"]) * np.log1p(np.abs(df_fundamentals_simplified_winsorize["Net Charge-Offs"]))

In [22]:
df_fundamentals_simplified_winsorize.head()

,datacqtr,datadate,rdq,tic,cik,Net Interest Income,Net Interest Margin,Net Charge-Offs,Cash and Short-Term Investments,Net Income
0,2000Q1,31/03/2000,19/04/2000,JPM,19617.0,7.799343,2.59,-6.376727,10.773106,7.595387
1,2000Q2,30/06/2000,19/07/2000,JPM,19617.0,7.748460,2.48,-6.359574,10.814585,7.398786
2,2000Q3,30/09/2000,18/10/2000,JPM,19617.0,7.767687,2.47,-6.327937,10.821617,7.243513
3,2000Q4,31/12/2000,17/01/2001,JPM,19617.0,7.816014,2.11,5.780744,11.530569,6.563856
4,2001Q1,31/03/2001,18/04/2001,JPM,19617.0,7.798523,2.11,-6.535241,11.527794,7.090077


In [82]:
df_fundamentals_simplified_winsorize["tic"].isna().sum()

0

In [99]:
df_fundamentals_simplified_winsorize["datacqtr"] = df_fundamentals_simplified_winsorize["datacqtr"].astype('period[Q-DEC]')

# Fundamentals - second file

In [24]:
df_bank_fundamentals_drive = pd.read_csv(f"{fundamentals_path}/banking_fundamental_drive.csv")
df_bank_fundamentals_drive["cik"] = df_bank_fundamentals_drive["cik"].astype(str)

/var/folders/78/9j91h4q516z7774553xzp19m0000gn/T/ipykernel_94817/3272942325.py:1: DtypeWarning: Columns (24,553,567,568,573,578) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bank_fundamentals_drive = pd.read_csv(f"{fundamentals_path}/banking_fundamental_drive.csv")


In [26]:
interested_cols_2 = ["dibq", "dpscq", "dptcq", "icaptq", "iditq"]
name_map_2 = {"dibq": "Deposits - Interest Bearing", "dpscq": "Total Savings Deposits", "dptcq": "Deposits - Total", "icaptq": "Invested Capital - Total", "iditq": "Interest Income - Total"}

# Deposits - Interest Bearing (dibq)
# Total Savings Deposits (dpscq)
# Deposits - Total (dptcq)
# Invested Capital - Total (icaptq)
# Interest Income - Total (iditq)
# Total Domestic Deposits (tdomdq)
# Total Foreign Deposits (tfdq)

# Total Current Operating Revenue ("tcorq")

In [27]:
df_bank_fundamentals_drive_simplified = df_bank_fundamentals_drive[key_cols + interested_cols_2 + ["tcorq"]]
df_bank_fundamentals_drive_simplified = df_bank_fundamentals_drive_simplified.rename(columns=name_map_2)
df_bank_fundamentals_drive_simplified = df_bank_fundamentals_drive_simplified.rename(columns={"tcorq": "Total Current Operating Revenue"})

In [28]:
df_bank_fundamentals_drive_simplified_win = df_bank_fundamentals_drive_simplified.copy()
limits_list_2 = [(0.01, 0.01), (0.01, 0.01), (0.01, 0.01), (0.01, 0.01), (0.01, 0.01)]
for col, limits in zip(name_map_2.values(), limits_list_2):
    df_bank_fundamentals_drive_simplified_win[col] = winsorize(df_bank_fundamentals_drive_simplified_win[col], limits=limits, nan_policy="omit")

In [29]:
for col in list(name_map_2.values())+["Total Current Operating Revenue"]:
    df_bank_fundamentals_drive_simplified_win[col] = np.sign(df_bank_fundamentals_drive_simplified_win[col]) * np.log1p(np.abs(df_bank_fundamentals_drive_simplified_win[col]))

In [79]:
df_bank_fundamentals_drive_simplified_win.head()

,datacqtr,datadate,rdq,tic,cik,Deposits - Interest Bearing,Total Savings Deposits,Deposits - Total,Invested Capital - Total,Interest Income - Total,Total Current Operating Revenue
0,2001Q1,03/31/2001,04/17/2001,ASO.1,3133.0,9.979315,9.317197,10.174482,9.116335,6.552247,6.783459
1,2001Q2,06/30/2001,07/17/2001,ASO.1,3133.0,9.964012,9.308687,10.165309,9.116410,6.521476,6.765302
2,2001Q3,09/30/2001,10/16/2001,ASO.1,3133.0,9.959834,9.339928,10.161031,9.119104,6.476001,6.735446
3,2001Q4,12/31/2001,01/15/2002,ASO.1,3133.0,9.946901,9.371591,10.172293,9.118483,6.411445,6.677961
4,2002Q1,03/31/2002,04/16/2002,ASO.1,3133.0,9.943832,9.386985,10.153226,9.120901,6.348362,6.619031


In [80]:
df_bank_fundamentals_drive_simplified_win["tic"].isna().sum()

64

In [98]:
df_bank_fundamentals_drive_simplified_win = df_bank_fundamentals_drive_simplified_win.dropna(subset=["tic"])
df_bank_fundamentals_drive_simplified_win["datacqtr"] = df_bank_fundamentals_drive_simplified_win["datacqtr"].astype('period[Q-DEC]')

# CAR

In [111]:
# Car

df_car = pd.read_excel("data/Car_Data.xlsx", sheet_name="Bank")

df_car['evtdate'] = pd.to_datetime(df_car['evtdate'])
df_car['datacqtr'] = df_car['evtdate'].dt.to_period('Q').astype('period[Q-DEC]')
df_car['tic'] = df_car['ticker'] 

df_car = df_car[["datacqtr", "tic", "car5"]]

In [112]:
df_car.head()

,datacqtr,tic,car5
0,2019Q2,TCF,0.016011
1,2018Q1,TCF,-0.022522
2,2018Q4,TCF,0.014032
3,2018Q2,TCF,0.041037
4,2018Q3,TCF,-0.033751


## Combine

In [138]:
dataframes = [merged_df_2000_to_2024, df_fundamentals_simplified_winsorize, df_bank_fundamentals_drive_simplified_win, df_car]

In [139]:
poc_data = pd.merge(df_bank_fundamentals_drive_simplified_win, df_fundamentals_simplified_winsorize, on=["datacqtr", "tic"], how="outer")
poc_data = poc_data.dropna(subset=["datacqtr"])

In [140]:
# poc_data["cik"].isna().sum()
# poc_data.dropna(subset=["tic"])

In [141]:
print(len(df_bank_fundamentals_drive_simplified_win) + len(df_fundamentals_simplified_winsorize))
print(len(poc_data))

81033
70532


In [142]:
poc_data

,datacqtr,datadate_x,rdq_x,tic,cik_x,Deposits - Interest Bearing,Total Savings Deposits,Deposits - Total,Invested Capital - Total,Interest Income - Total,Total Current Operating Revenue,datadate_y,rdq_y,cik_y,Net Interest Income,Net Interest Margin,Net Charge-Offs,Cash and Short-Term Investments,Net Income
31,2000Q1,NaN,NaN,ALNC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31/03/2000,14/04/2000,796317.0,NaN,4.36,-0.095310,3.321432,0.847584
32,2000Q1,NaN,NaN,AMNB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31/03/2000,19/04/2000,741516.0,1.844352,4.51,-0.062035,2.837498,1.136229
33,2000Q1,NaN,NaN,AROW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31/03/2000,17/04/2000,717538.0,2.381673,4.15,-0.138892,3.383169,1.430550
34,2000Q1,NaN,NaN,ASRV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31/03/2000,18/04/2000,707605.0,2.834154,2.75,-0.142367,3.781618,1.283985
35,2000Q1,NaN,NaN,BAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31/03/2000,17/04/2000,70858.0,8.428799,3.27,-6.042633,11.160684,7.714677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70558,2024Q4,12/31/2024,01/28/2025,WSBF,1569994.0,NaN,NaN,7.215899,6.194681,3.402929,3.892983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70559,2024Q4,12/31/2024,01/27/2025,WSFS,828944.0,NaN,NaN,9.742779,8.041884,5.577232,5.850831,31/12/2024,27/01/2025,828944.0,NaN,3.80,-2.415914,7.051026,4.177490
70560,2024Q4,12/31/2024,01/23/2025,WTBA,1166928.0,NaN,NaN,8.119278,6.431420,3.918323,3.968744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70561,2024Q4,12/31/2024,01/21/2025,WTFC,1015328.0,NaN,NaN,10.868823,9.281439,6.818379,6.938078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [143]:
poc_data_2 = pd.merge(poc_data, merged_df_2000_to_2024, on="datacqtr", how="outer")

In [144]:
poc_data_2

,datacqtr,datadate_x,rdq_x,tic,cik_x,Deposits - Interest Bearing,Total Savings Deposits,Deposits - Total,Invested Capital - Total,Interest Income - Total,...,Net Interest Margin,Net Charge-Offs,Cash and Short-Term Investments,Net Income,GDP CHANGE (-1 to 1),UNEMPLOYMENT RATE (0 to 1),PRIME LOAN RATE (0 to 1),DEPOSITS CHANGE (-1 to 1),CONSUMER PRICE INDEX (0 to 1),SAVINGS PER GROSS INCOME (-1 to 1)
0,2000Q1,NaN,NaN,ALNC,NaN,NaN,NaN,NaN,NaN,NaN,...,4.36,-0.095310,3.321432,0.847584,0.010304,0.040333,0.086944,0.019192,0.023993,0.063
1,2000Q1,NaN,NaN,AMNB,NaN,NaN,NaN,NaN,NaN,NaN,...,4.51,-0.062035,2.837498,1.136229,0.010304,0.040333,0.086944,0.019192,0.023993,0.063
2,2000Q1,NaN,NaN,AROW,NaN,NaN,NaN,NaN,NaN,NaN,...,4.15,-0.138892,3.383169,1.430550,0.010304,0.040333,0.086944,0.019192,0.023993,0.063
3,2000Q1,NaN,NaN,ASRV,NaN,NaN,NaN,NaN,NaN,NaN,...,2.75,-0.142367,3.781618,1.283985,0.010304,0.040333,0.086944,0.019192,0.023993,0.063
4,2000Q1,NaN,NaN,BAC,NaN,NaN,NaN,NaN,NaN,NaN,...,3.27,-6.042633,11.160684,7.714677,0.010304,0.040333,0.086944,0.019192,0.023993,0.063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70527,2024Q4,12/31/2024,01/28/2025,WSBF,1569994.0,NaN,NaN,7.215899,6.194681,3.402929,...,NaN,NaN,NaN,NaN,0.011879,0.041333,0.078231,0.009943,0.038620,0.006
70528,2024Q4,12/31/2024,01/27/2025,WSFS,828944.0,NaN,NaN,9.742779,8.041884,5.577232,...,3.80,-2.415914,7.051026,4.177490,0.011879,0.041333,0.078231,0.009943,0.038620,0.006
70529,2024Q4,12/31/2024,01/23/2025,WTBA,1166928.0,NaN,NaN,8.119278,6.431420,3.918323,...,NaN,NaN,NaN,NaN,0.011879,0.041333,0.078231,0.009943,0.038620,0.006
70530,2024Q4,12/31/2024,01/21/2025,WTFC,1015328.0,NaN,NaN,10.868823,9.281439,6.818379,...,NaN,NaN,NaN,NaN,0.011879,0.041333,0.078231,0.009943,0.038620,0.006


In [145]:
poc_data_3 = pd.merge(poc_data_2, df_car, on=["datacqtr", "tic"], how="outer")

In [146]:
poc_data_3 = poc_data_3.loc[(poc_data_3['datacqtr'] >= min_q) & (poc_data_3['datacqtr'] <= max_q)]

In [147]:
len(poc_data_3)

70783

In [148]:
len((poc_data_3['tic'] + poc_data_3['datacqtr'].astype(str)).unique())  # some duplicates

70696

In [149]:
poc_data_3 = poc_data_3[['datacqtr', 'tic',
       'Deposits - Interest Bearing', 'Total Savings Deposits',
       'Deposits - Total', 'Invested Capital - Total',
       'Interest Income - Total', 'Net Interest Income',
       'Net Interest Margin', 'Net Charge-Offs',
       'Cash and Short-Term Investments', 'Net Income', 'GDP CHANGE (-1 to 1)',
       'UNEMPLOYMENT RATE (0 to 1)', 'PRIME LOAN RATE (0 to 1)',
       'DEPOSITS CHANGE (-1 to 1)', 'CONSUMER PRICE INDEX (0 to 1)',
       'SAVINGS PER GROSS INCOME (-1 to 1)', 'Total Current Operating Revenue', 'car5']]

In [150]:
poc_data_3 = poc_data_3.rename(columns={'Total Current Operating Revenue': 'Y1 - Total Current Operating Revenue', 'car5': 'Y2 - car5'})

In [152]:
poc_data_3.to_csv("data/model_building_data.csv")